# Tutorial 3: Other Methods
In this tutorial, we'll describe how to incorporate other Lipschitz estimation techniques into our codebase. For each method, we try to use the official codebase attached to the papers. We simply built an interface on top of each of these to be more amenable to our system. The methods we consider are:
* **CLEVER**: Uses randomly sampled points and extremal value theory to generate an heuristic Lipschitz estimate([github](https://github.com/IBM/CLEVER-Robustness-Score)). 
* **FastLip**: Uses the hyperbox and boolean hyperbox abstract domains to efficiently generate an upper bound to the Lipschitz constant ([github](https://github.com/huanzhang12/CertifiedReLURobustness)).
* **LipLP**: The naive linear programming relaxation to LipMIP
* **LipSDP**: Uses incremental quadratic constraints and semidefinite programming to generate a global upper bound of Lipschitz constants in the $\ell_2$ setting ([github](https://github.com/arobey1/LipSDP)).
* **SeqLip**: Frames Lipschitz estimation as a combinatorial optimization problem and uses greedy methods to generate a heuristic Lipschitz estimate ([github](https://github.com/avirmaux/lipEstimation)).
* **RandomLB**: Randomly samples points and takes their maximal gradient norm. This is like CLEVER, but doesn't use the extremal value theory step, and thereby provides a certifiable lower bound.
* **NaiveUB**: Multiplies the operator norm of each component of a ReLU network together to yield an extremely loose upper bound.

Note that LipSDP uses the Mosek plugin for matlab to solve SDP's. Follow the instructions in their github to install these dependencies.

In [1]:
# Imports
# import matlab.engine

import sys
sys.path.append('..')
import torch 
from pprint import pprint 
import numpy as np

import utilities as utils
from relu_nets import ReLUNet
from hyperbox import Hyperbox 
from lipMIP import LipMIP
from other_methods import CLEVER, FastLip, LipLP, NaiveUB, RandomLB
import experiment as exp 

# 1: Individual Methods
The interface to run each method is identical, and all inherit a generic `OtherResult` class (except for LipMIP). We demonstrate how to run each method here.

Many methods have variants and hyperparameters that can be tuned. We incorporate these as kwargs and can tune them in our repository, but leave them mostly as default from their original codebases.

In [2]:
# Basic network example 

test_network = torch.load('2by2comparison.pt')
test_network = ReLUNet([2, 100, 100, 2], manual_net = test_network)
test_domain = Hyperbox.build_unit_hypercube(2)
primal_norm = 'linf'
c_vector = torch.Tensor([1, -1])

# 2: The `Experiment` class
As a convenient and flexible shorthand to evaluate lipschitz constants of various networks under various settings, we built the `Experiment` class which is very handy for performing common operations.

In [3]:
eval_class = [CLEVER, FastLip, LipLP, NaiveUB, RandomLB]

In [4]:
# --- build experiment object 
basic_exp = exp.Experiment(eval_class, network=test_network, c_vector=c_vector, primal_norm=primal_norm)

In [5]:
#  evaluating local lipschitz constants across [-r, +r]^d where r is a parameter taken to be large
T=[]
epsis=np.linspace(0.001, 1, num=10)
for epsilon in epsis:
    large_r_result = basic_exp.do_large_radius_eval(epsilon)
    T.append(large_r_result.values())
print(T)

[{'CLEVER': 0.05958084017038345, 'FastLip': 0.05958084, 'LipLP': 0.05958084017038345, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.0596)}, {'CLEVER': 0.15852293372154236, 'FastLip': 5.5080576, 'LipLP': 5.395470239583757, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.1247)}, {'CLEVER': 0.2652759552001953, 'FastLip': 7.424658, 'LipLP': 7.309843351300828, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.2649)}, {'CLEVER': 0.3093941807746887, 'FastLip': 9.269087, 'LipLP': 9.122953795347009, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.2899)}, {'CLEVER': 0.41606879234313965, 'FastLip': 10.033554, 'LipLP': 9.90973699257325, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.3963)}, {'CLEVER': 0.41606879234313965, 'FastLip': 11.18173, 'LipLP': 11.03768231066567, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.3963)}, {'CLEVER': 0.41606879234313965, 'FastLip': 12.409562, 'LipLP': 12.253464338085777, 'NaiveUB': 35.692245, 'RandomLB': tensor(0.4045)}, {'CLEVER': 0.41606879234313965, 'FastLip': 12.656469, 'LipLP': 12.5223

In [ ]:
eval_class = [LipMIP]
basic_exp = exp.Experiment(eval_class, network=test_network, c_vector=c_vector, primal_norm=primal_norm)
T=[]
epsis=np.linspace(0.001, 1, num=10)
for epsilon in epsis:
    large_r_result = basic_exp.do_large_radius_eval(epsilon)
    T.append(large_r_result.values())
print(T)